# N8n Workflow Scraper

This project automates scraping n8n workflows, saving JSON locally, uploading to Google Drive, and updating a Google Sheet with workflow data and Drive links, all via a Gradio interface.

In [ ]:
!pip install selenium gspread google-api-python-client google-auth pyperclip

In [9]:
#!/usr/bin/env python3
"""
Complete Browser Automation for n8n SEO Workflow Scraping
This script implements your exact workflow requirements:
1. Navigate to search results page
2. Click on workflow links (including pagination)
3. Extract workflow data (JSON, title, description, URL)
4. Save JSON locally
5. Upload JSON to Google Drive folder
6. Get Google Drive shareable link
7. Update Google Sheet with workflow data and Drive link
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time
import csv
import json
import re
import os
import pyperclip

# Import Google API libraries
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
import gspread
from google.auth import default # Assuming google.colab.auth.authenticate_user() is run before this script

class N8nBrowserAutomation:
    def __init__(self, headless=True, google_sheet_url=None, google_drive_folder_id=None, start_url=None):
        """Initialize the browser automation and Google API clients"""
        self.setup_chrome_driver(headless)
        self.workflows = []
        self.processed_urls = set()
        self.start_url = start_url # Store the start URL

        # Create output directory for local JSONs
        os.makedirs("automated_json_workflows", exist_ok=True)

        # Google Sheet and Drive setup
        self.google_sheet_url = google_sheet_url
        self.google_drive_folder_id = google_drive_folder_id
        self.sheet = None
        self.worksheet = None
        self.drive_service = None
        self.google_api_creds = None # To store credentials from google.auth.default()

        if self.google_sheet_url or self.google_drive_folder_id:
             try:
                 # Get credentials from google.colab.auth.authenticate_user()
                 self.google_api_creds, _ = default()
                 print("✓ Google API credentials obtained.")

                 if self.google_sheet_url:
                     # Initialize gspread client
                     self.gc = gspread.authorize(self.google_api_creds)
                     self.sheet = self.gc.open_by_url(self.google_sheet_url)
                     self.worksheet = self.sheet.get_worksheet(0) # Assuming first sheet
                     print(f"✓ Connected to Google Sheet: {self.google_sheet_url}")

                     # Prepare Sheet header if empty
                     if not self.worksheet.get_all_values():
                          header = ["title", "description", "url", "json_file_name", "drive link"]
                          self.worksheet.append_row(header)
                          print("✓ Added header row to Google Sheet.")

                 if self.google_drive_folder_id:
                     # Initialize Google Drive API client
                     self.drive_service = build('drive', 'v3', credentials=self.google_api_creds)
                     print(f"✓ Google Drive API service initialized for folder ID: {self.google_drive_folder_id}")

             except Exception as e:
                 print(f"✗ Failed to initialize Google API clients: {e}")
                 print("Please ensure google.colab.auth.authenticate_user() was run successfully and APIs are enabled.")


        # Workflow counts
        self.total_found = 0
        self.successful_extractions = 0
        self.successful_uploads = 0
        self.successful_sheet_updates = 0


    def setup_chrome_driver(self, headless=True):
        """Setup Chrome WebDriver with options"""
        chrome_options = Options()

        if headless:
            chrome_options.add_argument("--headless")

        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

        # Disable notifications and popups
        prefs = {
            "profile.default_content_setting_values.notifications": 2,
            "profile.default_content_settings.popups": 0,
            "profile.managed_default_content_settings.images": 2  # Don't load images for speed
        }
        chrome_options.add_experimental_option("prefs", prefs)

        try:
            self.driver = webdriver.Chrome(options=chrome_options)
            self.wait = WebDriverWait(self.driver, 20) # Increased wait time
            print("✓ Chrome WebDriver initialized successfully")
        except Exception as e:
            print(f"✗ Failed to initialize Chrome WebDriver: {e}")
            print("Make sure Chrome and ChromeDriver are installed")
            raise

    def sanitize_filename(self, filename):
        """Clean filename for safe file system usage"""
        filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
        filename = re.sub(r'\s+', '_', filename)
        if len(filename) > 100:
            filename = filename[:80] + "..." + filename[-17:]
        return filename.strip()

    def navigate_to_search_results(self):
        """Navigate to the SEO search results page"""
        url = self.start_url if self.start_url else "https://n8n.io/workflows/categories/crypto-trading/" # Use start_url if provided
        print(f"Navigating to: {url}")

        try:
            self.driver.get(url)
            time.sleep(3)  # Wait for page to load

            print(f"✓ Successfully loaded: {self.driver.title}")
            return True

        except Exception as e:
            print(f"✗ Failed to navigate to search results: {e}")
            return False

    def extract_workflow_links(self):
        """Extract workflow links from the current page"""
        workflow_links = []

        try:
            # Find all links that match workflow pattern
            links = self.driver.find_elements(By.CSS_SELECTOR, "a[href*='/workflows/']")

            for link in links:
                href = link.get_attribute('href')
                if href and re.match(r'.*/workflows/\d+.*', href):
                    if href not in workflow_links and href not in self.processed_urls:
                        workflow_links.append(href)

            # print(f"Found {len(workflow_links)} new workflow links on this page") # Suppress this print inside the loop for cleaner output
            return workflow_links

        except Exception as e:
            print(f"Error extracting workflow links: {e}")
            return []

    # Removed click_use_for_free_button()

    # Removed click_copy_to_clipboard()

    def extract_workflow_json(self):
        """Extract workflow JSON directly from <n8n-demo> element"""
        try:
            n8n_demo = self.wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "n8n-demo"))
            )
            workflow_json = n8n_demo.get_attribute("workflow")
            if workflow_json and len(workflow_json) > 100:
                print(f"✓ Successfully extracted workflow JSON ({len(workflow_json)} characters)")
                return workflow_json
            else:
                print("✗ Workflow JSON is empty or too short")
                return None
        except Exception as e:
            print(f"✗ Error extracting workflow JSON: {e}")
            return None


    def extract_workflow_metadata(self):
        """Extract title, description, and URL from current page"""
        try:
            # Extract title from og:title meta tag
            title_element = self.driver.find_element(By.CSS_SELECTOR, "meta[property='og:title']")
            title = title_element.get_attribute("content")

            # Extract description from og:description meta tag
            try:
                desc_element = self.driver.find_element(By.CSS_SELECTOR, "meta[property='og:description']")
                description = desc_element.get_attribute("content")
            except NoSuchElementException:
                description = title  # Fallback to title

            # Extract URL from og:url meta tag
            try:
                url_element = self.driver.find_element(By.CSS_SELECTOR, "meta[property='og:url']")
                url = url_element.get_attribute("content")
            except NoSuchElementException:
                url = self.driver.current_url  # Fallback to current URL

            return {
                'title': title,
                'description': description,
                'url': url
            }

        except Exception as e:
            print(f"Error extracting metadata: {e}")
            return {
                'title': 'Unknown Title',
                'description': 'No description available',
                'url': self.driver.current_url
            }

    def save_json_file_locally(self, json_content, title):
        """Save JSON content to file locally with sanitized title"""
        if not json_content:
            return None

        try:
            # Validate JSON content
            json_data = json.loads(json_content)

            filename = self.sanitize_filename(title) + ".json"
            filepath = os.path.join("automated_json_workflows", filename)

            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(json_data, f, indent=2, ensure_ascii=False)

            return filename

        except json.JSONDecodeError:
            print(f"✗ Invalid JSON content for {title}")
            return None
        except Exception as e:
            print(f"Error saving JSON file locally for {title}: {e}")
            return None

    def upload_json_to_drive(self, file_path):
        """Uploads a file to Google Drive folder and returns its shareable link."""
        if not self.drive_service or not self.google_drive_folder_id:
            print("✗ Google Drive service not initialized or folder ID not provided.")
            return None

        try:
            file_name = os.path.basename(file_path)
            media = MediaFileUpload(file_path, resumable=True)

            file_metadata = {'name': file_name, 'parents': [self.google_drive_folder_id]}

            # Upload the file
            uploaded_file = self.drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, webViewLink' # Request file ID and shareable link
            ).execute()

            file_id = uploaded_file.get('id')
            shareable_link = uploaded_file.get('webViewLink')

            print(f"✓ Uploaded '{file_name}' to Drive (ID: {file_id}).")

            # Make the file shareable (optional, based on user's statement "all are public")
            try:
                self.drive_service.permissions().create(
                    fileId=file_id,
                    body={'type': 'anyone', 'role': 'reader'},
                    fields='id'
                ).execute()
                print(f"✓ File '{file_name}' is now shareable.")
            except Exception as share_error:
                print(f"Warning: Could not make '{file_name}' shareable: {share_error}")


            self.successful_uploads += 1
            return shareable_link

        except Exception as e:
            print(f"✗ An error occurred during Drive upload for '{os.path.basename(file_path)}': {e}")
            return None

    def update_google_sheet(self, data_row):
        """Appends a row of data to the Google Sheet."""
        if not self.worksheet:
            print("✗ Google Sheet not initialized.")
            return False

        try:
            self.worksheet.append_row(data_row)
            print(f"✓ Appended data to Google Sheet for: {data_row[0]}")
            self.successful_sheet_updates += 1
            return True
        except Exception as e:
            print(f"✗ An error occurred while updating Google Sheet for: {data_row[0]}: {e}")
            return False


    def process_single_workflow(self, workflow_url):
        """Processes a single workflow: scrapes, saves locally, uploads to Drive, updates Sheet."""
        print(f"\n{'='*80}")
        print(f"Processing: {workflow_url}")
        print(f"{'='*80}")

        workflow_result = {
            'title': 'N/A',
            'description': 'N/A',
            'url': workflow_url, # Default to current URL
            'json_file_name': 'N/A',
            'drive_link': 'N/A'
        }

        try:
            # Step 1: Navigate to workflow page
            self.driver.get(workflow_url)
            time.sleep(2)

            # Step 2: Extract metadata (title, description, URL)
            metadata = self.extract_workflow_metadata()
            workflow_result['title'] = metadata['title']
            workflow_result['description'] = metadata['description']
            workflow_result['url'] = metadata['url']
            print(f"Title: {workflow_result['title']}")
            print(f"URL: {workflow_result['url']}")


            # Step 3: Directly extract JSON from <n8n-demo>
            json_content = self.extract_workflow_json()

            if not json_content:
                print("✗ Failed to extract workflow JSON.")
                # Still record metadata even if JSON extraction fails
                if self.worksheet:
                    self.update_google_sheet([
                        workflow_result['title'],
                        workflow_result['description'],
                        workflow_result['url'],
                        workflow_result['json_file_name'],
                        workflow_result['drive_link']
                    ])
                return workflow_result # Return result even if JSON extraction failed


            self.successful_extractions += 1

            # Step 4: Save JSON file locally
            json_filename = self.save_json_file_locally(json_content, workflow_result['title'])

            if not json_filename:
                print("✗ Failed to save JSON file locally.")
                 # Still record metadata even if local save fails
                if self.worksheet:
                    self.update_google_sheet([
                        workflow_result['title'],
                        workflow_result['description'],
                        workflow_result['url'],
                        workflow_result['json_file_name'],
                        workflow_result['drive_link']
                    ])
                return workflow_result # Return result even if local save failed


            workflow_result['json_file_name'] = json_filename
            local_file_path = os.path.join("automated_json_workflows", json_filename)
            print(f"✓ Saved JSON file locally: {local_file_path}")


            # Step 5 & 6: Upload JSON to Google Drive and get link
            if self.drive_service and self.google_drive_folder_id:
                 drive_link = self.upload_json_to_drive(local_file_path)
                 workflow_result['drive_link'] = drive_link if drive_link else 'N/A'
            else:
                print("Skipping Google Drive upload: Service not initialized or folder ID missing.")


            # Step 7: Update Google Sheet
            if self.worksheet:
                 self.update_google_sheet([
                     workflow_result['title'],
                     workflow_result['description'],
                     workflow_result['url'],
                     workflow_result['json_file_name'],
                     workflow_result['drive_link']
                 ])
            else:
                print("Skipping Google Sheet update: Service not initialized.")


            return workflow_result

        except Exception as e:
            print(f"✗ Error processing workflow {workflow_url}: {e}")
             # Attempt to log even failed workflows to the sheet if possible
            if self.worksheet:
                 try:
                      self.update_google_sheet([
                         workflow_result.get('title', 'Error'),
                         workflow_result.get('description', f'Error: {e}'),
                         workflow_result.get('url', workflow_url),
                         workflow_result.get('json_file_name', 'N/A'),
                         workflow_result.get('drive_link', 'N/A')
                      ])
                 except Exception as sheet_log_error:
                      print(f"✗ Failed to log error to sheet: {sheet_log_error}")
            return None
        finally:
            # Mark as processed
            self.processed_urls.add(workflow_url)


    def click_load_more_templates(self):
        """Click 'Load more templates' button if available"""
        try:
            load_more_selector = "button.search-results-load-more-btn"

            # Wait for the load more button to be clickable
            load_more_button = WebDriverWait(self.driver, 30).until( # Increased timeout
                EC.element_to_be_clickable((By.CSS_SELECTOR, load_more_selector))
            )

            # Scroll to button if needed
            self.driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
            time.sleep(1)

            # Click the button and wait for new content to load
            self.driver.execute_script("arguments[0].click();", load_more_button) # Use JavaScript click for reliability
            print("✓ Clicked 'Load more templates' button")
            time.sleep(5)  # Wait for new content to load

            # After clicking, wait for new workflow links to be present
            WebDriverWait(self.driver, 20).until( # Wait for at least one new link
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/workflows/']"))
            )
            return True

        except TimeoutException:
             print("Timed out waiting for 'Load more templates' button or button not clickable")
             return False
        except NoSuchElementException:
            print("No 'Load more templates' button found")
            return False
        except Exception as e:
            print(f"Error clicking load more button: {e}")
            return False

    def run_complete_automation(self):
        """Run the complete automation: scrape, upload, and update Sheet."""
        print("="*80)
        print("STARTING N8n SEO WORKFLOW AUTOMATION")
        print("Following process:")
        print("1. Navigate to search results page & load all links.")
        print("2. For each workflow: scrape data, save JSON locally, upload to Drive, update Sheet.")
        print("="*80)

        if not (self.google_sheet_url or self.google_drive_folder_id):
            print("Error: Google Sheet URL or Drive Folder ID not provided. Cannot proceed with Google integration.")
            print("Please provide at least one of these when initializing N8nBrowserAutomation.")
            return


        try:
            # Step 1: Initial navigation to search results
            if not self.navigate_to_search_results():
                 print("Failed to navigate to initial search results page. Aborting.")
                 return

            # Step 2: Extract all workflow links by loading more pages
            all_workflow_links = set()
            previous_count = 0
            page_num = 1

            while True:
                print(f"\n🔄 Loading and extracting links from page {page_num}...")
                current_links = self.extract_workflow_links()
                newly_found_links = [link for link in current_links if link not in all_workflow_links]

                if not newly_found_links:
                    print(f"No *new* workflow links found on page {page_num}.")
                    # If no *new* links are found, and we couldn't load more last time, break
                    print("Attempting to click 'Load more templates' to check for more pages...")
                    if not self.click_load_more_templates():
                        print("No more pages to load or failed to load more templates.")
                        break
                    else:
                        # If load more was clicked, wait for new content to render and try extracting links again
                        time.sleep(5) # Give the page time to render new links
                        page_num += 1
                        previous_count = len(all_workflow_links) # Update previous count after loading more
                        continue # Continue the loop to extract links from the new content


                all_workflow_links.update(newly_found_links)
                print(f"Found {len(newly_found_links)} new links. Total links collected: {len(all_workflow_links)}")

                # If the number of links didn't increase significantly after attempting to load more, break
                # This helps prevent infinite loops if click_load_more_templates succeeds but no new links appear
                if len(all_workflow_links) <= previous_count + 5 and page_num > 1: # Check if at least 5 new links were added (adjust threshold as needed)
                     print("Warning: Few or no new links found after clicking 'Load more'. Assuming end of results or issue.")
                     # Try one more click just in case, then break if still no significant change
                     if not self.click_load_more_templates():
                          print("No significant new links loaded and cannot load more. Assuming end of results.")
                          break
                     else:
                          # If another click worked, continue
                          time.sleep(5)
                          page_num += 1
                          previous_count = len(all_workflow_links)
                          continue


                previous_count = len(all_workflow_links)

                # Try clicking 'Load more templates' for the next page
                print(f"\n🔄 Attempting to load more templates for page {page_num + 1}...")
                if not self.click_load_more_templates():
                    print("No more pages to load")
                    break

                page_num += 1

                # Safety break to avoid excessive loops
                if page_num > 100: # Increased safety limit for pages
                    print("Reached maximum page limit for loading links.")
                    break


            self.total_found = len(all_workflow_links)
            print(f"\nFinished collecting links. Total unique workflows found: {self.total_found}")

            # Step 3: Process each collected workflow link (scrape, save, upload, update sheet)
            print(f"\n{'='*80}")
            print("STARTING WORKFLOW PROCESSING, UPLOAD, AND SHEET UPDATE")
            print(f"{'='*80}")

            if not all_workflow_links:
                 print("No workflows to process.")
                 return


            for i, workflow_url in enumerate(list(all_workflow_links), 1):
                 # Check if we have already processed this URL in case of multiple runs or partial runs
                 # This check is now less critical as we filter processed_urls when collecting links
                 # but can be kept as an extra safeguard if needed.
                 # if workflow_url in self.processed_urls:
                 #     print(f"\nSkipping already processed workflow: {workflow_url}")
                 #     continue

                 print(f"\n[{i}/{len(all_workflow_links)}] Processing workflow {self.successful_extractions + 1}...")

                 # Process the workflow, which now includes saving JSON, uploading, and updating the sheet
                 result = self.process_single_workflow(workflow_url)
                 if result:
                     # The result is already logged to the sheet within process_single_workflow
                     pass


                 # Brief pause between workflows
                 time.sleep(1)

            print(f"\n{'='*80}")
            print("AUTOMATION COMPLETE!")
            print(f"Total unique workflows identified: {self.total_found}")
            print(f"Successful JSON extractions: {self.successful_extractions}")
            print(f"Successful Drive uploads: {self.successful_uploads}")
            print(f"Successful Sheet updates: {self.successful_sheet_updates}")
            print(f"JSON files saved locally in: automated_json_workflows/")
            if self.google_drive_folder_id:
                 print(f"JSON files uploaded to Google Drive folder ID: {self.google_drive_folder_id}")
            if self.google_sheet_url:
                 print(f"Results updated in Google Sheet: {self.google_sheet_url}")
            print(f"{'='*80}")


        except KeyboardInterrupt:
            print("\n⚠️  Automation interrupted by user")
        except Exception as e:
            print(f"\n❌ Unexpected error during automation: {e}")
        finally:
            self.cleanup()

    def save_results_to_csv(self):
        """Removed: Results are now updated directly in Google Sheet."""
        pass # This method is no longer used


    def cleanup(self):
        """Clean up resources"""
        if hasattr(self, 'driver'):
            self.driver.quit()
            print("✓ Browser closed")


In [ ]:
!pip install gradio

In [10]:
import gradio as gr
from google.colab import auth
from google.auth import default

# Authenticate user for Google API access
# This will open a new tab for authentication
try:
    auth.authenticate_user()
    print('Authenticated')
except Exception as e:
    print(f"Authentication failed: {e}")
    print("Please ensure you have enabled the Google Drive and Google Sheets APIs for your project.")


def scrape_workflows_gradio(folder_link: str, sheet_link: str, start_url: str) -> str:
    """
    Initiates the N8n workflow scraping automation with Gradio inputs.

    Args:
        folder_link: The Google Drive folder ID.
        sheet_link: The Google Sheet URL.
        start_url: The starting URL for scraping.

    Returns:
        A string indicating the status of the scraping process.
    """
    try:
        # Instantiate the automation class with inputs from Gradio
        automation = N8nBrowserAutomation(
            headless=True, # Run in headless mode for Gradio app
            google_sheet_url=sheet_link,
            google_drive_folder_id=folder_link, # Use folder_link as the ID
            start_url=start_url # Pass the start URL
        )

        # Run the automation
        automation.run_complete_automation()

        return "Scraping process initiated and completed (check console for details)."

    except Exception as e:
        return f"An error occurred during scraping: {e}"

# Define the Gradio interface
interface = gr.Interface(
    fn=scrape_workflows_gradio,
    inputs=[
        gr.Textbox(label="Google Drive Folder ID"),
        gr.Textbox(label="Google Sheet URL"),
        gr.Textbox(label="Starting URL for Scraping")
    ],
    outputs="text",
    title="N8n Workflow Scraper",
    description="Enter your Google Drive Folder ID, Google Sheet URL, and the starting URL to scrape N8n workflows and save them to Drive and Sheet."
)

# Launch the Gradio app
interface.launch(debug=True)

Authenticated
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a797447cc9ca23d384.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✓ Chrome WebDriver initialized successfully
✓ Google API credentials obtained.
✓ Connected to Google Sheet: https://docs.google.com/spreadsheets/d/1ZymWkU7v8R0UhMyjfTZcvU1cn8zfS0bJVuQk4mdjpTk/edit
✓ Google Drive API service initialized for folder ID: 1OCattfy5XzdV2Ap5xUL1ibwcDFTyTemu
STARTING N8n SEO WORKFLOW AUTOMATION
Following process:
1. Navigate to search results page & load all links.
2. For each workflow: scrape data, save JSON locally, upload to Drive, update Sheet.
Navigating to: https://n8n.io/workflows/categories/SEO/
✓ Successfully loaded: 

🔄 Loading and extracting links from page 1...
No *new* workflow links found on page 1.
Attempting to click 'Load more templates' to check for more pages...
Timed out waiting for 'Load more templates' button or button not clickable
No more pages to load or failed to load more templates.

Finished collecting links. Total unique workflows found: 0

STARTING WORKFLOW PROCESSING, UPLOAD, AND SHEET UPDATE
No workflows to process.
✓ Browser cl